# Text Analytics Introduction

This notebook will explore the creation of a feature-vector representation of a text corpus using ideas from [here](http://nbviewer.ipython.org/github/jakevdp/sklearn_scipy2013/blob/master/notebooks/05.2_application_to_text_mining.ipynb).


In [ ]:
%matplotlib inline 

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import nltk.stem
from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 6)


## Simple Text Representation Example

Let's start with a simple example to illustrate common steps in creating a feature-vector representation of text.

### Pre-processing

First we'll convert the text into a `bag-of-words`. We can do this using `scikit`'s built-in `CountVectorizer`. The `min_df` parameter determines how the function will treat words that are used infrequently. If set to an interger, all words occuring less often will be dropped. If set to a fraction, all words that occur less frequently than that fraction of the overall dataset will be dropped.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=1)
vect

We see that for now the counting is done at the word level (`analyzer = word`).

Now let's create some text and use our `CountVectorizer` object.

In [ ]:
content = ['how to open a beer without a bottle opener', 
           'Beer bottles or beer cans',]
X = vect.fit_transform(content)

vect.get_feature_names()

Our transform "tokenized" the corpus, normalized everything to lowercase, and dropped words of length one.

Let's look at the vectorized word frequencies:

In [ ]:
print(X)

This information is stored in a memory-efficient coordinate matrix form. For analysis, we'll need to use feature vectors:

In [ ]:
print(X.toarray())

Now let's add some more data:

In [ ]:
posts = ['how to open a beer without a bottle opener', 
           'Do girls like beer bottles or beer cans?',
           'where did all my beer go?',
           'where did all my beer go? where did all my beer go?',
           'recycling beer bottles and cans',
           'Is it worth recycling?',
           'do not bring bottles to my backyard party, only cans please.', 
           'This is useless']

In [ ]:
X_train = vect.fit_transform(posts)

num_samples, num_features = X_train.shape

print('#samples: {}, #features: {}'.format(num_samples, num_features))

There are eight posts in our corpus with a total of 31 different words. Now we can vectorize the data.

Let's vectorize a new post then see how similar it is to our corpus.

In [ ]:
new_post = 'Opening beer bottles and cans 101'
new_post_vect = vect.transform([new_post])

print(new_post_vect.toarray())

In [ ]:
print(X_train.toarray())

We'll use the Euclidean "norm" to measure the similarity between two vectors. 

For convenience, let's define a couple of functions first.

In [ ]:
def dists(v1, v2):
    delta = v1-v2
    # Calculate Euclidean "norm" distance
    return sp.linalg.norm(delta.toarray())

import sys

def similarity(new_post_vector, corpus):
    best_dist = 999
    best_i = None
    
    for i in range(len(corpus.toarray())):
        post = posts[i]
        
        if post == new_post:
            continue
        post_vec = corpus.getrow(i)
        d = dists(post_vec, new_post_vector)
        print('Post %i with dist = %.2f: %s'%(i, d, post))
        
        if d < best_dist:
            best_dist = d
            best_i = i
    print('Best post is {} with dist = {}'.format(best_i, best_dist))

In [ ]:
similarity(new_post_vect, X_train)

Great, our first text similarity measurement!

Notice that Post 3 is least similar to our new post. Post 2 is more similar even though Post 3 is just Post 2 doubled. It is clear the simple counts of words is too simple. The next step is to normalize the word counts to avoid this problem.

Let's update our distance function:

In [ ]:
def dists(v1, v2):
    v1_norm = v1/sp.linalg.norm(v1.toarray())
    v2_norm = v2/sp.linalg.norm(v2.toarray())
    delta = v1_norm-v2_norm
    # Calculate Euclidean "norm" distance
    return sp.linalg.norm(delta.toarray())

In [ ]:
similarity(new_post_vect, X_train)

Great! Now Post 2 and Post 3 have the same result.

### Removing less important words

There are many words in language that do not carry much meaning in terms of the overall interpretation of the message. Words like "it" should be much less meaningful than "beer" in our current context. These less important words are called "stop words", and can be removed from the posts since they do not help us distiguish between different posts.

Let's add English stop words to our vectorizer then look at a sample of them:

In [ ]:
vect = CountVectorizer(min_df=1, stop_words='english')

print(sorted(vect.get_stop_words())[80:-150])

If you already have a list of words in mind that you wish to stop, you can simply pass them as a list to the stop_words argument.

### Stemming

We also need to consider that similar words, such as "girl" and "girls" should probably be considered as the same word. Thus we need a function that reduces words to a finite 'word stem'. We can do this with the **Natural Language Toolkit (NLTK)**. 

Let's try it:

In [ ]:
s = nltk.stem.SnowballStemmer('english')

print(s.stem('bottles'))
print(s.stem('bottle'))

print(s.stem('perception'))
print(s.stem('perceptive'))

print(s.stem('crashing'))
print(s.stem('crashed'))

### Extending the vectorizer with NLTK stemming

We need to step the posts before we feed then into the `CountVectorizer`. The best way to do this is overwrite the method `build_analyzer`. 

By doing this we utilize the preprocessing functions in the parent class that converts the raw posts into lower case. We tokenize all the words, and then convert each word into the stemmed version.

For those who are not familiar with methods and classes, this might not be clear. Don't worry - just do it!

In [ ]:
english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')

In [ ]:
X = vectorizer.fit_transform(posts)
vectorizer.get_feature_names()

Restate the `X_train` vectorizer on the new data and check similarity for `new_post_vect`.

In [ ]:
X_train = vectorizer.fit_transform(posts)
new_post_vect = vectorizer.transform([new_post])

similarity(new_post_vect, X_train)

Well ... that made a difference! 

We see now that Post 0 is most similar to our new post, because bottles and bottle are now treated as the same word.

In [ ]:
print(new_post)
print(posts[0])

### Thinking a bit deeper about relevant post features

So far we have considered that higher occurrence of certains words in a post equates to a greater importance of that word in the post. While this is true to some extent, there is the case where very frequent words really don't carry any meaning to posts. For example, if the word "Subject" appears in every blog post, then it is not really communicating anything important and doesn't help us distinguish between posts.

We could perhaps set a 90% occurrence cutoff in our tokenizer, such that words that occur in >90% of the posts are excluded, however, we still run into the problem of border cases, say where the word occurs in only 89% of the posts.

To solve these problems we count the term frequencies for each post **while** discounting those words that appear in many posts. This is the concept of **term frequency - inverse document frequency (TF-IDF)**. We can implement this using `scikit-learn`'s `TfidfVectorizer`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Rebuild the function to include our stemmer

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english', decode_error='ignore')

Now instead of counts, our document vectors will contain individual TF-IDF values per term (token).

Again, restate the vectorizer:

In [ ]:
X_train = vectorizer.fit_transform(posts)
new_post_vect = vectorizer.transform([new_post])

similarity(new_post_vect, X_train)

## Recap

So far we have:

1. Tokenized text
2. Discarded words that occur too often and don't help us detect relevant posts
3. Thrown away very uncommon words
4. Counted the remaining words
5. Calculated TF-IDF values from the counts, considering the whole text corpus.

**Limitations of the bag-of-words approach**

* It does not cover word relations: "Car hits wall" and "Wall hits car" will both have the same feature vector.
* It does not count negations well: "I will eat soup" and "I will *not* eat soup" will have very similar feature vectors. This can be remedied by also counting bigrams and trigrams (two or three words in a row together) or n-grams.
* Totally fails with misspelled words.

## Text Analytics on Newsgroup Posts

We will:

- Turn a corpus of text documents into **feature vectors** using a **Bag of Words** representation
- Train a simple text classifier on the feature vectors
- Check the classifier accuracy

### The dataset

We will use a "canonical" text classification dataset from the official website for [20 Newsgroups Dataset](http://qwone.com/~jason/20Newsgroups/). The data is organized into 20 different newsgroups, each corresponding to a different topic. There are three versions of the data. We'll use the one that has 18,846 documents sorted by date into training(60%) and test(40%) sets.

The data available are in .tar.gz bundles. For BA575, the 14 MB "20news-bydate.tar" file is on Canvas. You should put this in the "data" subdirectory then expand it (simple on a Mac). This will create two subdirectories under "data" - one for training and one for test. This Notebook assumes those two subdirectories are first level "data" subdirectories (i.e., data/20news-bydate-train/ and data/20news-bydate-test/.

We'll be using `scikit-learn` to load the data. [Here](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/datasets/twenty_newsgroups.py) is the official `scikit-learn` Python script that can be used for fetching the data from the official website.

### Text Classification in 20 Lines of Python Code

We will explore all of this in more detail below. But let's just see how simply the workflow really can be!

In just 20 lines of Python code, we will:

- Load the 20 newsgroups dataset: 18,846 text posts from 20 newsgroups forums
- Create Bag of Words features-vector representation with TF-IDF weighting
- Use a Naive Bayes classifier

In [ ]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load the text data
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
twenty_train_subset = load_files('data/20news-bydate-train/',
    categories=categories, encoding='latin-1')
twenty_test_subset = load_files('data/20news-bydate-test/',
    categories=categories, encoding='latin-1')

# Turn the text documents into vectors of word frequencies
vectorizer = TfidfVectorizer(min_df=2)
X_train = vectorizer.fit_transform(twenty_train_subset.data)
y_train = twenty_train_subset.target

# Fit a classifier on the training set
classifier = MultinomialNB().fit(X_train, y_train)
print("Training score: {0:.1f}%".format(
    classifier.score(X_train, y_train) * 100))

# Evaluate the classifier on the testing set
X_test = vectorizer.transform(twenty_test_subset.data)
y_test = twenty_test_subset.target
print("Testing score: {0:.1f}%".format(
    classifier.score(X_test, y_test) * 100))

#### Impressive!

Here's a workflow diagram summary of what happened:

<img src="supervised_scikit_learn.png">

If you'd like to learn **a lot** more about the details, go [here](http://nbviewer.ipython.org/github/jakevdp/sklearn_scipy2013/blob/master/notebooks/05.2_application_to_text_mining.ipynb).